In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf
import keras

tsv_file = '/Users/krishnapaparaju/Downloads/train.tsv'
train_data_df = pd.read_table(tsv_file)

Using TensorFlow backend.


In [2]:
train_data_df

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2
5,6,1,of escapades demonstrating the adage that what...,2
6,7,1,of,2
7,8,1,escapades demonstrating the adage that what is...,2
8,9,1,escapades,2
9,10,1,demonstrating the adage that what is good for ...,2


In [3]:
sentiments = train_data_df['Sentiment'].values
sentences = train_data_df['Phrase'].values

In [4]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [5]:
sia = SentimentIntensityAnalyzer()

def polarity(phrase):
    return list(sia.polarity_scores(phrase).values())

In [6]:
polarities = []

for i in range(len(sentences)):
    polarities.append(polarity(sentences[i]))

polarities = np.array(polarities)

In [7]:
max_features = 16467
# cut texts after this number of words
# (among top max_features most common words)
maxlen = 55
batch_size = 32

In [8]:
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical

tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(sentences)
sequences = tokenizer.texts_to_sequences(sentences)

sentiments = to_categorical(sentiments)

In [9]:
from numpy import asarray
import os

embeddings_index = {}
f = open(os.path.join('/Users/krishnapaparaju/Downloads', 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    if values[0] == 'ehow':
        word = values[1]
        coefs = np.asarray(values[2:], dtype='float32')
    else:
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [10]:
word_index = tokenizer.word_index

embedding_matrix = np.zeros((len(word_index) + 1, 100))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [11]:
train_features = sequences[:134848]
train_polarities = polarities[:134848]
train_targets = sentiments[:134848]

val_features = sequences[134848:]
val_polarities = polarities[134848:]
val_targets = sentiments[134848:]

In [12]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Embedding, GRU, Bidirectional

In [13]:
print('Pad sequences (samples x time)')
train_features = sequence.pad_sequences(train_features, maxlen=maxlen, padding='pre')
print('train_features shape:', train_features.shape)
train_features = np.array(train_features)

Pad sequences (samples x time)
('train_features shape:', (134848, 55))


In [14]:
print('Pad sequences (samples x time)')
val_features = sequence.pad_sequences(val_features, maxlen=maxlen, padding='pre')
print('val_features shape:', val_features.shape)
val_features = np.array(val_features)

Pad sequences (samples x time)
('val_features shape:', (21212, 55))


In [15]:
from keras.models import *
from keras.layers import *
from keras.initializers import *
from keras.optimizers import *

In [16]:
maxlen = 55
max_features = 15289
embed_size = 100

In [17]:
# https://www.kaggle.com/suicaokhoailang/lstm-attention-baseline-0-652-lb

class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

In [18]:
def squash(x, axis=-1):
    # s_squared_norm is really small
    # s_squared_norm = K.sum(K.square(x), axis, keepdims=True) + K.epsilon()
    # scale = K.sqrt(s_squared_norm)/ (0.5 + s_squared_norm)
    # return scale * x
    s_squared_norm = K.sum(K.square(x), axis, keepdims=True)
    scale = K.sqrt(s_squared_norm + K.epsilon())
    return x / scale

# A Capsule Implement with Pure Keras
class Capsule(Layer):
    def __init__(self, num_capsule, dim_capsule, routings=3, kernel_size=(9, 1), share_weights=True,
                 activation='default', **kwargs):
        super(Capsule, self).__init__(**kwargs)
        self.num_capsule = num_capsule
        self.dim_capsule = dim_capsule
        self.routings = routings
        self.kernel_size = kernel_size
        self.share_weights = share_weights
        if activation == 'default':
            self.activation = squash
        else:
            self.activation = Activation(activation)

    def build(self, input_shape):
        super(Capsule, self).build(input_shape)
        input_dim_capsule = input_shape[-1]
        if self.share_weights:
            self.W = self.add_weight(name='capsule_kernel',
                                     shape=(1, input_dim_capsule,
                                            self.num_capsule * self.dim_capsule),
                                     # shape=self.kernel_size,
                                     initializer='glorot_uniform',
                                     trainable=True)
        else:
            input_num_capsule = input_shape[-2]
            self.W = self.add_weight(name='capsule_kernel',
                                     shape=(input_num_capsule,
                                            input_dim_capsule,
                                            self.num_capsule * self.dim_capsule),
                                     initializer='glorot_uniform',
                                     trainable=True)

    def call(self, u_vecs):
        if self.share_weights:
            u_hat_vecs = K.conv1d(u_vecs, self.W)
        else:
            u_hat_vecs = K.local_conv1d(u_vecs, self.W, [1], [1])

        batch_size = K.shape(u_vecs)[0]
        input_num_capsule = K.shape(u_vecs)[1]
        u_hat_vecs = K.reshape(u_hat_vecs, (batch_size, input_num_capsule,
                                            self.num_capsule, self.dim_capsule))
        u_hat_vecs = K.permute_dimensions(u_hat_vecs, (0, 2, 1, 3))
        # final u_hat_vecs.shape = [None, num_capsule, input_num_capsule, dim_capsule]

        b = K.zeros_like(u_hat_vecs[:, :, :, 0])  # shape = [None, num_capsule, input_num_capsule]
        for i in range(self.routings):
            b = K.permute_dimensions(b, (0, 2, 1))  # shape = [None, input_num_capsule, num_capsule]
            c = K.softmax(b)
            c = K.permute_dimensions(c, (0, 2, 1))
            b = K.permute_dimensions(b, (0, 2, 1))
            outputs = self.activation(tf.keras.backend.batch_dot(c, u_hat_vecs, [2, 2]))
            if i < self.routings - 1:
                b = tf.keras.backend.batch_dot(outputs, u_hat_vecs, [2, 3])

        return outputs

    def compute_output_shape(self, input_shape):
        return (None, self.num_capsule, self.dim_capsule)

In [19]:
def BiGRUCapsNet():
    inp = Input(shape=(maxlen,))
    f = Input(shape=(4,))
    
    embedding = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp)
    spatial_dropout = SpatialDropout1D(rate=0.2)(embedding)
    
    bi_gru_1 = Bidirectional(GRU(64, return_sequences=True, 
                             kernel_initializer=glorot_normal(seed=12300), recurrent_initializer=orthogonal(gain=1.0, seed=10000)))(spatial_dropout)
    
    bi_gru_2 = Bidirectional(GRU(64, return_sequences=True, 
                             kernel_initializer=glorot_normal(seed=12300), recurrent_initializer=orthogonal(gain=1.0, seed=10000)))(bi_gru_1)

    capsule = Capsule(num_capsule=10, dim_capsule=10, routings=4, share_weights=True)(bi_gru_1)
    capsule = Flatten()(capsule)
    
    gru_atten_1 = Attention(maxlen)(bi_gru_1)
    gru_atten_2 = Attention(maxlen)(bi_gru_2)

    avg_pool = GlobalAveragePooling1D()(bi_gru_2)
    max_pool = GlobalMaxPooling1D()(bi_gru_2)
    
    features = concatenate([gru_atten_1, gru_atten_2, capsule, avg_pool, max_pool, f])
    outp = Dense(5, activation='softmax')(features)

    model = Model(inputs=[inp, f], outputs=outp)
    model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['acc'])
    return model

In [20]:
model = BiGRUCapsNet()
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 55)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 55, 100)      1528900     input_1[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_1 (SpatialDro (None, 55, 100)      0           embedding_1[0][0]                
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 55, 128)      63360       spatial_dropout1d_1[0][0]        
__________________________________________________________________________________________________
bidirectio

In [21]:
model.fit(x=[train_features, train_polarities], y=train_targets, epochs=5, batch_size=128, validation_data=([val_features, val_polarities], val_targets))

Train on 134848 samples, validate on 21212 samples
Epoch 1/5
134848/134848 [==============================] - 232s 2ms/step - loss: 0.9846 - acc: 0.5937 - val_loss: 0.9645 - val_acc: 0.5945
Epoch 2/5
134848/134848 [==============================] - 219s 2ms/step - loss: 0.9077 - acc: 0.6230 - val_loss: 0.9372 - val_acc: 0.6037
Epoch 3/5
134848/134848 [==============================] - 221s 2ms/step - loss: 0.8721 - acc: 0.6380 - val_loss: 0.9257 - val_acc: 0.6134
Epoch 4/5
134848/134848 [==============================] - 197s 1ms/step - loss: 0.8433 - acc: 0.6496 - val_loss: 0.9210 - val_acc: 0.6152
Epoch 5/5
134848/134848 [==============================] - 279s 2ms/step - loss: 0.8220 - acc: 0.6605 - val_loss: 0.9158 - val_acc: 0.6181


In [22]:
tsv_file = '/Users/krishnapaparaju/Downloads/test.tsv'
test_data_df = pd.read_table(tsv_file)

In [23]:
test_data = test_data_df.values

In [24]:
sentences = test_data_df['Phrase'].values

In [25]:
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical

test_sequences = tokenizer.texts_to_sequences(sentences)
test_sequences = np.array(sequence.pad_sequences(test_sequences, maxlen=maxlen))

In [26]:
test_polarities = []

for i in range(len(sentences)):
    test_polarities.append(polarity(sentences[i]))
    
test_polarities = np.array(test_polarities)

In [30]:
predictions = model.predict([test_sequences, test_polarities])

class_predictions = []

for i in range(0, len(predictions)):
    class_predictions.append(list.index(list(predictions[i]), max(predictions[i])))

In [28]:
ids = list(test_data[:, 0])
submission = pd.DataFrame(np.transpose(np.array([ids, class_predictions])))
submission.columns = ['PhraseId', 'Sentiment']
# submission.to_csv('Movie-Review-Sentiment-Predictions-1.csv', index=False)

In [31]:
submission.head(20)

,PhraseId,Sentiment
0,156061,2
1,156062,2
2,156063,2
3,156064,2
4,156065,2
5,156066,3
6,156067,3
7,156068,2
8,156069,3
9,156070,2
